# Notebook 4: Masked Modeling (BERT, MAE)

In this notebook, we delve into masked modeling, a strategy that has become dominant in NLP with models like BERT, and has recently shown great promise in vision with Masked Autoencoders (MAE). Masked modeling is essentially a form of “fill-in-the-blank” self-supervised task: you mask/hide part of the input and train the model to predict the missing content.

## Masked Language Modeling (BERT)

BERT (Bidirectional Encoder Representations from Transformers), introduced by Devlin et al. (2018), popularized masked language modeling (MLM). The idea: take a sentence, mask out some words, and train a transformer to predict those masked words from the context. By doing so on billions of words, BERT learns deep semantic and syntactic understanding of language without any explicit labels. How it works:
- During pre-processing, 15% of the tokens in a text sequence are selected for masking. For each selected token:
  - 80% of the time it is replaced with a [MASK] token (a placeholder symbol not seen in normal text).
  - 10% of the time it's replaced with a random word.
  - 10% of the time it is left unchanged (this helps the model not to rely solely on the presence of [MASK], mitigating distribution shift at inference).
- The model is a multi-layer bidirectional Transformer encoder that processes the entire sequence with masked tokens in place, and tries to predict the original token at each masked position. This is usually done via a softmax over the vocabulary (so it’s a classification among all words for each mask).
- Additionally, BERT had a secondary task called Next Sentence Prediction (NSP) where the model learns to predict if one sentence follows another, but MLM is the primary driver of its representation learning.

Why is MLM powerful? Because to fill in a blank, the model must use context from both left and right of the position – it forces a deep understanding of how words relate. For example, in the sentence "The cat sat on the [MASK].", possible predictions could be "mat" (likely) or "sofa" etc., but the model must consider grammar and context. BERT, through MLM, ends up learning word meanings, part-of-speech, even some world knowledge, all as a byproduct of this fill-in task. Training: BERT is trained on massive corpora (Wikipedia + BooksCorpus, ~3.3 billion words) for many epochs. It converges to a point where the predictions are quite accurate. For instance, if you feed a trained BERT the sentence "Paris is the [MASK] of France.", it will strongly predict "capital" at the mask (since it learned factual knowledge during training).

Let's see an example using a pre-trained BERT model via Hugging Face transformers to do mask filling:


In [ ]:
!pip install transformers
from transformers import pipeline
mask_filler = pipeline("fill-mask", model="bert-base-uncased")
result = mask_filler("The capital of France is [MASK].")
print(result)

This will output something like:
```python
[{'sequence': 'the capital of france is paris.', 
  'score': 0.97, 'token': 3000, 'token_str': 'paris'}]
```

BERT correctly predicts "Paris" with high confidence. This demonstrates the knowledge it picked up. We can try another:

In [ ]:
result = mask_filler("A [MASK] a day keeps the doctor away.")
print([r['sequence'] for r in result])

Possible output:
```vbnet
['a apple a day keeps the doctor away.',
 'a pill a day keeps the doctor away.',
 'a medication a day keeps the doctor away.',
 'a drink a day keeps the doctor away.',
 'a banana a day keeps the doctor away.']
```

## Masked Image Modeling (MAE, BEiT, etc.)

Applying the same concept to images took some time to become popular. Early attempts (like Pathak’s Context Encoders in 2016) did “image inpainting” – mask a part of the image and reconstruct it – but results weren’t as groundbreaking as BERT for text. However, with the advent of Vision Transformers (ViT) which operate on patches of images similarly to words, researchers revisited masked modeling for images. Masked Autoencoders (MAE) by He et al. (2021) demonstrated that a simple masked image reconstruction can yield very powerful image features. The setup:
- Divide the image into patches (ViT patch tokens).
- Mask a very high fraction of them (e.g., 75% masking, so only 25% patches visible).
- An encoder (ViT) processes only the visible patches.
- A lightweight decoder then takes the encoded visible patches plus mask tokens as placeholders for the missing patches, and tries to reconstruct the original image pixels for the masked patches.

The task: predict the pixel values of the masked patches (could be done in pixel space or some compressed space).

MAE uses an MSE loss between the reconstructed and original pixels for the masked patches. By randomly masking and reconstructing, the model learns to encode images such that it can predict missing parts. Important aspects:
- **High mask ratio:** They masked 75% (and even up to 90% in some experiments). This made the task sufficiently challenging – if too little is masked, the task might be trivial (like colorization where structure is intact) but with huge parts missing, the model must truly understand global structure.
- **Asymmetric design:** The encoder only sees visible patches (so it's efficient), and the decoder is small and only used during pre-training. After pre-training, you throw away the decoder and just use the encoder (which has now learned strong features).

The intuition: if you can reconstruct a coherent image from seeing just 25% of patches, you must have learned what the object or scene is. For example, if an image of a car is largely masked except a wheel and a headlight, the model can guess there is a car and fill in the rest.

Results: MAE pre-training on ImageNet, followed by fine-tuning, produced state-of-the-art classification results. Notably, a ViT-Huge with MAE pre-training surpassed 87% top-1 on ImageNet, higher than previous contrastive or supervised methods with similar architecture.  MAE architecture. The image is split into patches. A large random subset (e.g., 75%) is masked (grey patches). The encoder operates on the remaining visible patches (colored). Mask tokens (grey squares) are then added and a decoder reconstructs the image, predicting pixels for masked patches. The decoder is removed after pre-training.

Masked image modeling is also done in other ways:
- **BEiT (2021):** Instead of pixels, BEiT tokenizes image patches into discrete tokens (using a dVAE). The task is to predict the discrete token of masked patches (like predicting a “visual word”). This is analogous to BERT but for images. BEiT showed good results as well, proving the concept of BERT-ifying images.
- **iGPT (2020):** OpenAI’s idea of GPT for images – treat image pixels as a sequence and predict pixels auto-regressively – was an earlier attempt at generative pretraining for images. It worked but not as well as contrastive or masked-patch approaches, partially because pixel-level prediction on full images is very hard.
- **Masked Feature Modeling:** Some approaches mask in the feature space of a pre-trained autoencoder or use teacher networks (e.g., Data2Vec, or MSN) so that the task is to predict some high-level representation rather than raw pixels.
  
Why masked modeling now? Vision transformers made it easier to drop patches and still have a meaningful processing (CNNs could also do it but not as straightforward to implement random masking with arbitrary patterns). Also, the community realized that simpler objectives like pixel reconstruction can work if done at scale with the right model capacity.

## Hands-on with Masked Modeling

For NLP, we saw how to use a pre-trained BERT for MLM. For images, let's use a pre-trained MAE model to reconstruct an image to see what it does. Hugging Face provides a MAE model (ViT architecture) trained on ImageNet. We can use it to reconstruct masked images:


In [ ]:
!pip install transformers
from transformers import AutoImageProcessor, AutoModelForMaskedImageModeling
from PIL import Image
import requests
import torch

# Load a pre-trained MAE
processor = AutoImageProcessor.from_pretrained("facebook/vit-mae-base")
model = AutoModelForMaskedImageModeling.from_pretrained("facebook/vit-mae-base")

# Load an example image
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/datasets/cats.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image = image.resize((224, 224))  # model expects 224x224
inputs = processor(images=image, return_tensors="pt")

# Generate mask labels (simulate masking as done in training)
# The model expects a mask already applied, so let's use its processing function
pixel_values = inputs["pixel_values"]
# We create a random mask - in MAE, this is done inside model during training.
# But for inference, the HuggingFace model's forward can accept a mask.
num_patches = (model.config.image_size // model.config.patch_size) ** 2  # e.g., (224/16)^2 = 196 patches
mask = torch.rand(num_patches) < 0.75  # mask 75% of patches
outputs = model(pixel_values, bool_masked_pos=mask)
reconstruction = outputs.logits  # reconstructed pixel values for patches
# Convert reconstruction to image
recon_img = processor.post_process(reconstruction, (224, 224))[0]
Image.fromarray(recon_img.astype('uint8')).save("reconstructed.png")

Instead of running this fully (which depends on the model), I'll describe the expected outcome: The reconstructed image should look like the original cat image, but perhaps slightly smoother or with some artifacts especially where there was heavy masking. MAE papers show that the reconstruction is not perfect when 75% masked, but still captures the gist (the cat’s shape, fur color, etc. appear correctly, just some detail is smoothed). What matters for representation learning is not the output fidelity per se, but that the encoder’s latent space now contains information to complete the image. That means the encoder has to understand objects in the image. When we fine-tune this encoder for classification, it’s already endowed with knowledge of what various objects generally look like.

**Fine-tuning MAE:** After pre-training, one typically attaches a classification head and trains on labeled data. MAE’s authors found that fine-tuning a ViT-B on ImageNet after MAE pre-training gave substantially better results than training ViT-B from scratch (which normally underperforms ResNet if done from scratch on not-so-huge data).

**More Masked Modeling Examples**
- **Segment Anything (SAM)** by Meta AI (2023) used a form of masked autoencoding on image embeddings for segmentation tasks (though not exactly the same, it predicts masked image regions).
- In audio, masked acoustic modeling is used (e.g., in models like Wav2Vec 2.0) where portions of the audio waveform or features are masked and predicted.

So masked modeling is a general SSL approach: images, text, audio, video, you name it. It is a form of denoising autoencoder (an older term): you corrupt the input and train a model to denoise it. Vincent et al., 2008 had introduced denoising autoencoders – today’s BERT and MAE are essentially sophisticated denoising autoencoders with transformers.


## Conclusion

Masked modeling marries the simplicity of “fill in missing data” with the power of big models and big data. It has become a dominant approach because:
- It doesn’t require designing tricky augmentations or negative pairs (like contrastive methods do).
- It directly utilizes the structure of data (sequential or spatial).
- It can scale with model size easily (BERT-large, ViT-Huge, etc.).

In the next notebook, we'll shift our focus to multimodal self-supervision with CLIP, which aligns images and text. But remember that CLIP’s image encoder could itself benefit from masked pre-training, or the text encoder is essentially a product of masked language modeling. These techniques can be combined.

**Bonus Exercise:** If you were to mask parts of a video, how would you design a masked video modeling task? One idea: mask random patches in space and time (e.g., a block of 16x16 pixels for 10 frames) and predict those. What might the model learn? Think about temporal consistency and object motion – masked modeling could force the model to learn dynamics as well as appearance.


## References:
- Devlin, J. et al. (2018). "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding." NAACL. – Introduced MLM and NSP pre-training for transformers.
- Liu, K. et al. (2019). "RoBERTa: A Robustly Optimized BERT Pretraining Approach." – Showed that with more data and no NSP, BERT variants get even better; underlines the importance of MLM.
- He, K. et al. (2021). "Masked Autoencoders Are Scalable Vision Learners." arXiv. – MAE paper, showing state-of-the-art results with masked image modeling.
- Bao, H. et al. (2021). "BEiT: BERT Pre-Training of Image Transformers." ICLR. – First to successfully apply BERT-style masking to vision (using a tokenizer on image patches).
- Pathak, D. et al. (2016). "Context Encoders: Feature Learning by Inpainting." CVPR. – Early work on image inpainting as SSL (CNNs context encoder).
- Chen, X. et al. (2020). "Generative Pretraining From Pixels." ICML. – The iGPT paper from OpenAI, an autoregressive approach to image modeling.
- Baevski, A. et al. (2020). "wav2vec 2.0: A Framework for Self-Supervised Learning of Speech Representations." NeurIPS. – Example of masked modeling in audio (mask segments of speech and predict them).
- Zhou, H. et al. (2021). "Data2Vec: A General Framework for Self-Supervised Learning in Speech, Vision and Language." arXiv. – Unified view of masked prediction across modalities.
